# import 

In [ ]:
%cd drive/MyDrive/CS506/Midterm
%ls 

/content/drive/MyDrive/CS506/Midterm
all_train.csv               new_test.csv      test_processed.csv
catboost_info/              sample.csv        train.csv
data.csv                    sample_train.csv  train+val.csv
data_processed.csv          submission.csv    trainval.csv
glove.6B.300d.txt           tes.csv
glove.6B.300d.word2vec.txt  test.csv


In [ ]:
!pip install lightgbm
!pip install gensim
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from concurrent.futures import ProcessPoolExecutor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import gensim.downloader as api
nltk.download('stopwords')
nltk.download('wordnet')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#load

In [ ]:
data = pd.read_csv('train.csv')
data.dropna(inplace=True)
data['index'] = list(range(len(data)))
data.head()

,Unnamed: 0,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,...,char_count,word_count,flesch_reading_ease,smog_index,gunning_fog,noun_count,verb_count,adj_count,adv_count,index
0,0,1049849,B000MR9D5E,A1EKSETIBS9ETQ,0,0,1198281600,"Great nature series, but not all scenes looked...",I have watched numbers of nature shows and ser...,4.0,...,547,82,-20.22,0.0,37.68,37,1,22,5,0
1,1,999834,B000GAKFIG,AR0HFYHYHDGQQ,2,5,1245024000,Agatha Christie's Marple: Series 2,As a devoted fan of all of Agatha Christie's f...,5.0,...,361,51,11.25,0.0,27.46,22,2,14,4,1
2,2,218826,6300215776,A37S3ACL57LN62,11,15,1126137600,Childish Entertainment,This movie is about a script writer and a secr...,2.0,...,757,112,-42.21,0.0,49.09,55,4,20,5,2
3,3,796384,B00019071C,A1TO1P3NV7OAU6,2,2,1351036800,The weakest Babylon 5 season,This is the weakest Babylon 5 season. After wi...,4.0,...,408,59,-5.34,0.0,27.67,32,0,11,4,3
4,4,1219784,B001NFNFN0,ATCM1W7HWIC6U,0,0,1381708800,Versatile and effective,This video will always have a sweet spot on my...,5.0,...,676,92,-47.29,0.0,42.45,32,1,24,5,4


In [ ]:
data.to_csv('train.csv',index=False)

In [ ]:
apposV2 = {
"are not" : "are not",
"ca" : "can",
"could n't" : "could not",
"did n't" : "did not",
"does n't" : "does not",
"do n't" : "do not",
"had n't" : "had not",
"has n't" : "has not",
"have n't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"is n't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"might n't" : "might not",
"must n't" : "must not",
"sha" : "shall",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"should n't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"were n't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"wo" : "will",
"would n't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"was n't": "was not",
"we'll":"we will",
"did n't": "did not"
}
appos = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}

In [ ]:
     split = [apposV2[word] if word in apposV2 else word for word in words] #correct using apposV2 as mentioned above
        split = [appos[word] if word in appos else word for word in split] #correct using appos as mentioned above

#preprocessing

In [ ]:
# Clean text function
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = text.split()
    text = [apposV2[word] if word in apposV2 else word for word in text if word not in set(stopwords.words('english'))]
    text = [appos[word] if word in appos else word for word in text]
    text = ' '.join(text)
    return text

# Function to process a chunk of the dataset
def process_chunk(chunk):
    return chunk.apply(clean_text)

# Function to parallelize the data processing using multiprocessing
def parallelize_dataframe_processing(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    with ProcessPoolExecutor(max_workers=n_cores) as executor:
        df = pd.concat(executor.map(func, df_split))
    return df

# Set the number of cores for multiprocessing
n_cores = 10

# Clean review titles and content using multiprocessing
data['cleaned_review_title'] = parallelize_dataframe_processing(data['Summary'], process_chunk, n_cores)
data['cleaned_review_content'] = parallelize_dataframe_processing(data['Text'], process_chunk, n_cores)

# Combine cleaned review titles and content
data['cleaned_combined_review'] = data['cleaned_review_title'] + ' ' + data['cleaned_review_content']


In [ ]:
data['Review'] = data['cleaned_combined_review']

In [ ]:
data = data[['Id', 'ProductId', 'UserId', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Time', 'Summary', 'Text', 'Score','Review']]
data.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Review
0,1224650,B001OQCV6A,A3HUN3KP383B23,0,1,1395619200,Sherlock Holmes,I bought this thinking it was Sherlock Holmes ...,NaN,sherlock holmes bought thinking sherlock holme...
1,1019381,B000I8OFLO,A2VJ80PM1G00QV,2,2,1222300800,"FINE SINGING, DESTRUCTIVE STAGING",1. Scenic design and the Directors use of it p...,NaN,fine singing destructive staging 1 scenic desi...
2,504719,B00000IQCC,A2EGK0YRDF4ZZB,11,11,1018310400,Moving story of Jesus's message in a modern re...,J&eacute;sus de Montr&eacute;al was a stunni...,NaN,moving story jesuss message modern reincarnati...
3,1622425,B00B5UBDA0,A21I62TCDL4754,0,0,1372118400,Paul Anka has aged well and performs well,I was looking for Blu Ray concerts for my new ...,NaN,paul anka aged well performs well looking blu ...
4,482286,6305892946,A1IQ9E6I3PIUFF,1,2,969321600,Great end to the series.Or is it???????????,Despite what everyone else says this movie is ...,NaN,great end seriesor despite everyone else says ...


In [ ]:
data.to_csv('new_data.csv',index=False)

In [ ]:
data[['cleaned_combined_review','Score']].to_csv('sample_train.csv',index=False)

#train

In [ ]:
sample = pd.read_csv('sample_train.csv')
sample.head()

,cleaned_combined_review,Score
0,great nature series scene looked highdefi watc...,4.0
1,agatha christie marple series 2 devoted fan ag...,5.0
2,childish entertainment movie script writer sec...,2.0
3,weakest babylon 5 season weakest babylon 5 sea...,4.0
4,versatile effective video always sweet spot sh...,5.0


In [ ]:
import pandas as pd

# Select rows with user_ids in the list
tes = data[data['Id'].isin(test_id)]
tes.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Review,...,smog_index,gunning_fog,noun_count,verb_count,adj_count,adv_count,vader_pos,vader_neg,vader_neu,vader_compound


In [ ]:
data = pd.read_csv('data_processed.csv').dropna()
data.head()

In [ ]:
data.to_csv('data_processed.csv',index=False)

In [ ]:
data.columns

Index(['Id', 'ProductId', 'UserId', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Time', 'Summary', 'Text', 'Score',
       'Helpfulness', 'ReviewLength', 'Review', 'polarity', 'subjectivity',
       'char_count', 'word_count', 'flesch_reading_ease', 'smog_index',
       'gunning_fog', 'noun_count', 'verb_count', 'adj_count', 'adv_count',
       'vader_pos', 'vader_neg', 'vader_neu', 'vader_compound'],
      dtype='object')

In [ ]:
print(len(data))

125777


In [ ]:
data.to_csv('train+val.csv',index=False)
tes.to_csv('tes.csv',index=False)

In [ ]:
data = pd.read_csv('trainval.csv')
tes = pd.read_csv('new_test.csv')

In [ ]:
data.to_csv('trainval.csv',index=False)
tes.to_csv('new_test.csv',index=False)

## LGBM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming `data` is your original DataFrame with columns 'UserId', 'ProductId', 'Rating'
# Split the dataset into train and test sets
train_data, test_data = train_test_split(data, test_size=0.00001, random_state=42)
# 0.0001
# Calculate the average rating per user and per product on the training set
user_avg = train_data.groupby('UserId')['Score'].mean()
product_avg = train_data.groupby('ProductId')['Score'].mean()

# Calculate the global average rating for users and products
global_user_average = user_avg.mean()
global_product_average = product_avg.mean()

# Define a function to get the average rating for a user or a product
def get_average_rating(row, avg_ratings, global_avg):
    if row in avg_ratings:
        return avg_ratings[row]
    else:
        return global_avg

# Apply the average ratings to the train and test sets
train_data['UserAvgRating'] = train_data['UserId'].apply(lambda x: get_average_rating(x, user_avg, global_user_average))
train_data['ProductAvgRating'] = train_data['ProductId'].apply(lambda x: get_average_rating(x, product_avg, global_product_average))

test_data['UserAvgRating'] = test_data['UserId'].apply(lambda x: get_average_rating(x, user_avg, global_user_average))
test_data['ProductAvgRating'] = test_data['ProductId'].apply(lambda x: get_average_rating(x, product_avg, global_product_average))

tes['UserAvgRating'] = tes['UserId'].apply(lambda x: get_average_rating(x, user_avg, global_user_average))
tes['ProductAvgRating'] = tes['ProductId'].apply(lambda x: get_average_rating(x, product_avg, global_product_average))

In [ ]:
from sklearn.model_selection import train_test_split

# Select the features and target variable
features = ['ProductId','UserId','HelpfulnessNumerator', 'HelpfulnessDenominator', 'Helpfulness', 'Review', 'polarity', 'subjectivity',
            'char_count', 'word_count', 'flesch_reading_ease', 'smog_index', 'gunning_fog', 'noun_count', 'verb_count', 'adj_count',
            'adv_count', 'vader_pos', 'vader_neg', 'vader_neu', 'vader_compound', 'UserAvgRating', 'ProductAvgRating']
target = 'Score'

X_train, X_test = train_data[features], test_data[features]
y_train, y_test = train_data[target], test_data[target]


In [ ]:
from sklearn.preprocessing import StandardScaler

# Select the non-text features
non_text_features = ['HelpfulnessNumerator', 'HelpfulnessDenominator', 'Helpfulness', 'polarity', 'subjectivity',
                     'char_count', 'word_count', 'flesch_reading_ease', 'smog_index', 'gunning_fog', 'noun_count', 'verb_count',
                     'adj_count', 'adv_count', 'vader_pos', 'vader_neg', 'vader_neu', 'vader_compound']

# Instantiate the scaler
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train[non_text_features])

# Transform the training and test data
X_train_non_text_scaled = scaler.transform(X_train[non_text_features])
X_test_non_text_scaled = scaler.transform(X_test[non_text_features])
x_test_non_text_scaled = scaler.transform(tes[non_text_features])

In [ ]:
def tokenize_corpus(corpus):
    return [text.lower().split() for text in corpus]

# Compute the Word2Vec embeddings for each document
def get_document_embeddings(tokenized_doc, word2vec_model):
    doc_embeddings = np.zeros(word2vec_model.vector_size)
    word_count = 0
    for word in tokenized_doc:
        if word in word2vec_model:
            doc_embeddings += word2vec_model[word]
            word_count += 1
    if word_count > 0:
        doc_embeddings /= word_count
    return doc_embeddings

In [ ]:
# Load the Word2Vec model
word2vec_model = api.load("word2vec-google-news-300")

# Tokenize the cleaned review text
tokenized_train_corpus = tokenize_corpus(train_data['Review'])
tokenized_test_corpus = tokenize_corpus(test_data['Review'])
tokenized_tes_corpus = tokenize_corpus(tes['Review'])

# Compute the Word2Vec embeddings for each document separately for train and test sets
train_word2vec_embeddings = np.array([get_document_embeddings(doc, word2vec_model) for doc in tokenized_train_corpus])
test_word2vec_embeddings = np.array([get_document_embeddings(doc, word2vec_model) for doc in tokenized_test_corpus])
tes_word2vec_embeddings = np.array([get_document_embeddings(doc, word2vec_model) for doc in tokenized_tes_corpus])



In [ ]:
int(1e6)

1000000

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate the vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_features=int(3e4))
# Fit the vectorizer on the training data
vectorizer.fit(X_train['Review'])

# Transform the training and test data
X_train_tfidf = vectorizer.transform(X_train['Review'])
X_test_tfidf = vectorizer.transform(X_test['Review'])
x_test_tfidf = vectorizer.transform(tes['Review'])

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
import scipy
# Extract topics using LDA
num_topics = 20
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
train_topic_distributions = lda_model.fit_transform(X_train_tfidf)
test_topic_distributions = lda_model.transform(X_test_tfidf)
tes_topic_distributions = lda_model.transform(x_test_tfidf)



In [ ]:
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv('data.csv')
# assuming df is your dataframe
enc = OneHotEncoder()
enc = enc.fit(df[['ProductId','UserId']])
train_enc = enc.transform(train_data[['ProductId','UserId']])
test_enc = enc.transform(test_data[['ProductId','UserId']])
tes_enc = enc.transform(tes[['ProductId','UserId']])

In [ ]:
import numpy as np
import scipy
# Convert the non-text features to sparse matrices
X_train_non_text_sparse = scipy.sparse.csr_matrix(X_train_non_text_scaled)
X_test_non_text_sparse = scipy.sparse.csr_matrix(X_test_non_text_scaled)
x_test_non_text_sparse = scipy.sparse.csr_matrix(x_test_non_text_scaled)

# Combine the TF-IDF and non-text features

# X_train_combined = scipy.sparse.hstack([X_train_tfidf, train_topic_distributions, X_train_non_text_sparse, train_enc])
# X_test_combined = scipy.sparse.hstack([X_test_tfidf, test_topic_distributions, X_test_non_text_sparse, test_enc])
# x_test_combined = scipy.sparse.hstack([x_test_tfidf, tes_topic_distributions, x_test_non_text_sparse, tes_enc])

X_train_combined = scipy.sparse.hstack([X_train_tfidf, X_train_non_text_sparse, train_enc])
X_test_combined = scipy.sparse.hstack([X_test_tfidf, X_test_non_text_sparse, test_enc])
x_test_combined = scipy.sparse.hstack([x_test_tfidf, x_test_non_text_sparse, tes_enc])

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

lgb_regressor = lgb.LGBMRegressor(random_state=42,num_leaves=120,learning_rate=0.15)

lgb_regressor.fit(X_train_combined, y_train)


# Make predictions on the test set
y_pred = lgb_regressor.predict(X_test_combined)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error: {mse:.4f}")

Mean squared error: 4.5449


non-topic  

1e4: .

5e4: .

1e5: .5915

5e5: .5934

1e6: .5898

3e6: .5941

5e6: .5981

In [ ]:
test = pd.read_csv('test.csv')
tes_pred = lgb_regressor.predict(x_test_combined)
test['Score'] = tes_pred
test.head()
test[['Id','Score']].to_csv('submission.csv',index=False)


## tfidf

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sample["cleaned_combined_review"], sample["Score"], test_size=0.2, random_state=42)
# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:


# Assuming you have already split the data and transformed it into TF-IDF features (X_train_tfidf, X_test_tfidf)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear', C=1, random_state=42)
#‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)
svm_pred = y_pred
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Print classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)



Accuracy: 59.70%
Confusion Matrix:
 [[ 40   6   7  14  60]
 [ 11   5  19  23  50]
 [  4   2  42  87 104]
 [  3   2  14 152 288]
 [  0   1   5 106 955]]
Classification Report:
               precision    recall  f1-score   support

         1.0       0.69      0.31      0.43       127
         2.0       0.31      0.05      0.08       108
         3.0       0.48      0.18      0.26       239
         4.0       0.40      0.33      0.36       459
         5.0       0.66      0.90      0.76      1067

    accuracy                           0.60      2000
   macro avg       0.51      0.35      0.38      2000
weighted avg       0.56      0.60      0.55      2000



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import AdaBoostClassifier

all = pd.read_csv('all_train.csv')[:60000]
# Create the CountVectorizer
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all["cleaned_combined_review"], all["Score"], test_size=0.2, random_state=42)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [ ]:
nb = MultinomialNB(alpha = 0.001,fit_prior = False)
nb_classifier = AdaBoostClassifier(estimator=nb, n_estimators=20, learning_rate = 1)
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Print classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

Accuracy: 57.45%
Confusion Matrix:
 [[  15    5    4   14   89]
 [   4    2    7   17   78]
 [   0    0   19   51  169]
 [   0    1    5   81  372]
 [   1    0    0   34 1032]]
Classification Report:
               precision    recall  f1-score   support

         1.0       0.75      0.12      0.20       127
         2.0       0.25      0.02      0.03       108
         3.0       0.54      0.08      0.14       239
         4.0       0.41      0.18      0.25       459
         5.0       0.59      0.97      0.74      1067

    accuracy                           0.57      2000
   macro avg       0.51      0.27      0.27      2000
weighted avg       0.54      0.57      0.48      2000



In [ ]:
y_pred.shape

(2000,)

In [ ]:
y_test

6252    5.0
4684    3.0
1731    5.0
4742    5.0
4521    5.0
       ... 
6412    5.0
8285    4.0
7853    1.0
1095    4.0
6929    5.0
Name: Score, Length: 2000, dtype: float64

In [ ]:
combine = pd.DataFrame(svm_pred,columns=['svm'])
combine['bayes']=y_pred
combine['Score']=[i for i in y_test]
combine.head()

,svm,bayes,Score
0,5.0,5.0,5.0
1,4.0,5.0,3.0
2,5.0,5.0,5.0
3,5.0,5.0,5.0
4,5.0,5.0,5.0


In [ ]:
tree = DecisionTreeClassifier(max_depth=l0, min_samples_split=5, min_samples_leaf=5)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load dataset
data = pd.read_csv('all_train.csv')[:20000]

# Create the CountVectorizer
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(data["cleaned_combined_review"], data["Score"], test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
X_val = vectorizer.transform(X_val)


In [ ]:

# Train base models (SVM and MultinomialNB)
svm_classifier = SVC(kernel='linear', C=1, random_state=42)
nb = MultinomialNB()
nb_classifier = AdaBoostClassifier(estimator=nb, n_estimators=10, learning_rate = 1)

svm_classifier.fit(X_train, y_train)
nb_classifier.fit(X_train, y_train)

# Get base model predictions for the validation set
svm_val_preds = svm_classifier.predict(X_val)
nb_val_preds = nb_classifier.predict(X_val)

# Combine base model predictions
stacked_val_preds = np.column_stack((svm_val_preds, nb_val_preds))

# Train the meta-model (Logistic Regression)
meta_model = LogisticRegression(random_state=42)
meta_model.fit(stacked_val_preds, y_val)

# Get base model predictions for the test set
svm_test_preds = svm_classifier.predict(X_test)
nb_test_preds = nb_classifier.predict(X_test)

# Combine base model predictions
stacked_test_preds = np.column_stack((svm_test_preds, nb_test_preds))

# Make final predictions using the meta-model
final_preds = meta_model.predict(stacked_test_preds)



In [ ]:
final_preds

array([5., 1., 5., ..., 4., 5., 5.])

In [ ]:

# Calculate the accuracy of the ensemble model
print(accuracy_score(y_test, svm_test_preds))
print(accuracy_score(y_test, nb_test_preds))
accuracy = accuracy_score(y_test, final_preds)
print(f"Ensemble Model Accuracy: {accuracy * 100:}%")

0.60775
0.55125
Ensemble Model Accuracy: 60.8%


##glove

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
glove_input_file = 'glove.6B.300d.txt'  # Update the file path accordingly
word2vec_output_file = 'glove.6B.300d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
glove_vectors = KeyedVectors.load_word2vec_format('glove.6B.300d.word2vec.txt', binary=False)

In [ ]:
# Function to calculate the average GloVe vector for a given text
def avg_glove_vector(text):
    words = text.split()
    word_vectors = [glove_vectors[word] for word in words if word in glove_vectors.key_to_index]
    
    if len(word_vectors) == 0:
        return np.zeros(glove_vectors.vector_size)
    else:
        return np.mean(word_vectors, axis=0)



In [ ]:
# Calculate the average GloVe vector for each movie review
sample['glove_features'] = sample['cleaned_combined_review'].apply(avg_glove_vector)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sample["glove_features"].tolist(), sample["Score"], test_size=0.2, random_state=42)

##SVM

In [ ]:
X_train.shape

(8000,)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming you have already split the data and transformed it into TF-IDF features (X_train_tfidf, X_test_tfidf)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear', C=1, random_state=42)
#‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Print classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

ValueError: ignored

# test process

In [ ]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
train.dropna(inplace=True)
all = pd.read_csv('all_train.csv')
test.head()

,Id,Score
0,1224650,NaN
1,1019381,NaN
2,504719,NaN
3,1622425,NaN
4,482286,NaN


In [ ]:
train.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score
0,1049849,B000MR9D5E,A1EKSETIBS9ETQ,0,0,1198281600,"Great nature series, but not all scenes looked...",I have watched numbers of nature shows and ser...,4.0
1,999834,B000GAKFIG,AR0HFYHYHDGQQ,2,5,1245024000,Agatha Christie's Marple: Series 2,As a devoted fan of all of Agatha Christie's f...,5.0
2,218826,6300215776,A37S3ACL57LN62,11,15,1126137600,Childish Entertainment,This movie is about a script writer and a secr...,2.0
3,796384,B00019071C,A1TO1P3NV7OAU6,2,2,1351036800,The weakest Babylon 5 season,This is the weakest Babylon 5 season. After wi...,4.0
4,1219784,B001NFNFN0,ATCM1W7HWIC6U,0,0,1381708800,Versatile and effective,This video will always have a sweet spot on my...,5.0


In [ ]:
sum(1 for i in train['Id'] if type(i)!=np.int64)

125775

In [ ]:
for num in all['Id']:
  if type(num) != np.int64:
    print(num)

In [ ]:
all['Id'] = [i for i in train['Id']]
all.head()

,cleaned_combined_review,Score,Id
0,great nature series scene looked highdefi watc...,4.0,1049849
1,agatha christie marple series 2 devoted fan ag...,5.0,999834
2,childish entertainment movie script writer sec...,2.0,218826
3,weakest babylon 5 season weakest babylon 5 sea...,4.0,796384
4,versatile effective video always sweet spot sh...,5.0,1219784


In [ ]:
d_id2review = {}
def create_dic(data):
  id,review = data
  d_id2review[id] = review
all[['Id','cleaned_combined_review']].apply(create_dic,axis=1)

In [ ]:
raw_train = pd.read_csv('train.csv')

In [ ]:
raw_train[raw_train['Id']==1224650]

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score
125777,1224650,B001OQCV6A,A3HUN3KP383B23,0,1,1395619200,Sherlock Holmes,I bought this thinking it was Sherlock Holmes ...,NaN


In [ ]:
d_id2review

In [ ]:
raw_train[raw_train['Id']==1224650]['Summary'].iloc[0]

'Sherlock Holmes'

In [ ]:
raw_train[raw_train['Id']==1224650]['Summary'].iloc[0]

'Sherlock Holmes'

In [ ]:
test['summary'] = [ raw_train[raw_train['Id']==id]['Summary'].iloc[0] for id in test['Id']]
test['Text'] = [ raw_train[raw_train['Id']==id]['Text'].iloc[0] for id in test['Id']]
test.head()

,Id,Score,summary,Text
0,1224650,NaN,Sherlock Holmes,I bought this thinking it was Sherlock Holmes ...
1,1019381,NaN,"FINE SINGING, DESTRUCTIVE STAGING",1. Scenic design and the Directors use of it p...
2,504719,NaN,Moving story of Jesus's message in a modern re...,J&eacute;sus de Montr&eacute;al was a stunni...
3,1622425,NaN,Paul Anka has aged well and performs well,I was looking for Blu Ray concerts for my new ...
4,482286,NaN,Great end to the series.Or is it???????????,Despite what everyone else says this movie is ...


In [ ]:
# Clean text function
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text if word not in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text

# Function to process a chunk of the dataset
def process_chunk(chunk):
    return chunk.apply(clean_text)

# Function to parallelize the data processing using multiprocessing
def parallelize_dataframe_processing(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    with ProcessPoolExecutor(max_workers=n_cores) as executor:
        df = pd.concat(executor.map(func, df_split))
    return df

# Set the number of cores for multiprocessing
n_cores = 10

# Clean review titles and content using multiprocessing
test['cleaned_review_title'] = parallelize_dataframe_processing(test['summary'], process_chunk, n_cores)
test['cleaned_review_content'] = parallelize_dataframe_processing(test['Text'], process_chunk, n_cores)

# Combine cleaned review titles and content
test['combined'] = test['cleaned_review_title'] + ' ' + test['cleaned_review_content']


In [ ]:
test.head()

,Id,Score,summary,Text,cleaned_review_title,cleaned_review_content,combined
0,1224650,NaN,Sherlock Holmes,I bought this thinking it was Sherlock Holmes ...,sherlock holmes,bought thinking sherlock holmes ii found wasnt...,sherlock holmes bought thinking sherlock holme...
1,1019381,NaN,"FINE SINGING, DESTRUCTIVE STAGING",1. Scenic design and the Directors use of it p...,fine singing destructive staging,1 scenic design director use posed enormous pr...,fine singing destructive staging 1 scenic desi...
2,504719,NaN,Moving story of Jesus's message in a modern re...,J&eacute;sus de Montr&eacute;al was a stunni...,moving story jesus message modern reincarnation,jeacutesus de montreacuteal stunning thoughtpr...,moving story jesus message modern reincarnatio...
3,1622425,NaN,Paul Anka has aged well and performs well,I was looking for Blu Ray concerts for my new ...,paul anka aged well performs well,looking blu ray concert new home theater syste...,paul anka aged well performs well looking blu ...
4,482286,NaN,Great end to the series.Or is it???????????,Despite what everyone else says this movie is ...,great end seriesor,despite everyone else say movie great dvd grea...,great end seriesor despite everyone else say m...


In [ ]:
test.to_csv('test_processed.csv',index=False)

# submission

In [ ]:
test = pd.read_csv('test_processed.csv')
X_Test = test['combined']
X_Test_tfidf = vectorizer.transform(X_Test)
y_pred = svm_classifier.predict(X_Test_tfidf)
test['Score'] = y_pred
new = test[['Id','Score']]
new.head()
new.to_csv('submission.csv',index=False)

# Dataset further extraction

In [ ]:
df = pd.read_csv('all_train.csv')
df.head()

,cleaned_combined_review,Score
0,great nature series scene looked highdefi watc...,4.0
1,agatha christie marple series 2 devoted fan ag...,5.0
2,childish entertainment movie script writer sec...,2.0
3,weakest babylon 5 season weakest babylon 5 sea...,4.0
4,versatile effective video always sweet spot sh...,5.0


In [ ]:
data = pd.read_csv('data.csv')
data.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Helpfulness,ReviewLength,Review
0,1049849,B000MR9D5E,A1EKSETIBS9ETQ,0,0,1198281600,"Great nature series, but not all scenes looked...",I have watched numbers of nature shows and ser...,4.0,0.000000,165,great nature series scene looked highdefi watc...
1,999834,B000GAKFIG,AR0HFYHYHDGQQ,2,5,1245024000,Agatha Christie's Marple: Series 2,As a devoted fan of all of Agatha Christie's f...,5.0,0.400000,77,agatha christie marple series devoted fan agat...
2,218826,6300215776,A37S3ACL57LN62,11,15,1126137600,Childish Entertainment,This movie is about a script writer and a secr...,2.0,0.733333,222,childish entertainment movie script writer sec...
3,796384,B00019071C,A1TO1P3NV7OAU6,2,2,1351036800,The weakest Babylon 5 season,This is the weakest Babylon 5 season. After wi...,4.0,1.000000,110,weakest babylon season weakest babylon season ...
4,1219784,B001NFNFN0,ATCM1W7HWIC6U,0,0,1381708800,Versatile and effective,This video will always have a sweet spot on my...,5.0,0.000000,190,versatile effective video always sweet spot sh...


In [ ]:
from textblob import TextBlob

def sentiment_features(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    return polarity, subjectivity

data['polarity'], data['subjectivity'] = zip(*data['Text'].apply(sentiment_features))
data.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Review,polarity,subjectivity
0,1224650,B001OQCV6A,A3HUN3KP383B23,0,1,1395619200,Sherlock Holmes,I bought this thinking it was Sherlock Holmes ...,NaN,sherlock holmes bought thinking sherlock holme...,0.500000,0.800000
1,1019381,B000I8OFLO,A2VJ80PM1G00QV,2,2,1222300800,"FINE SINGING, DESTRUCTIVE STAGING",1. Scenic design and the Directors use of it p...,NaN,fine singing destructive staging 1 scenic desi...,0.225000,0.551923
2,504719,B00000IQCC,A2EGK0YRDF4ZZB,11,11,1018310400,Moving story of Jesus's message in a modern re...,J&eacute;sus de Montr&eacute;al was a stunni...,NaN,moving story jesuss message modern reincarnati...,0.104454,0.523987
3,1622425,B00B5UBDA0,A21I62TCDL4754,0,0,1372118400,Paul Anka has aged well and performs well,I was looking for Blu Ray concerts for my new ...,NaN,paul anka aged well performs well looking blu ...,0.468182,0.602273
4,482286,6305892946,A1IQ9E6I3PIUFF,1,2,969321600,Great end to the series.Or is it???????????,Despite what everyone else says this movie is ...,NaN,great end seriesor despite everyone else says ...,0.800000,0.750000


In [ ]:
data['char_count'] = data['Review'].apply(len)
data['word_count'] = data['Review'].apply(lambda x: len(x.split()))
data.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Review,polarity,subjectivity,char_count,word_count
0,1224650,B001OQCV6A,A3HUN3KP383B23,0,1,1395619200,Sherlock Holmes,I bought this thinking it was Sherlock Holmes ...,NaN,sherlock holmes bought thinking sherlock holme...,0.500000,0.800000,176,24
1,1019381,B000I8OFLO,A2VJ80PM1G00QV,2,2,1222300800,"FINE SINGING, DESTRUCTIVE STAGING",1. Scenic design and the Directors use of it p...,NaN,fine singing destructive staging 1 scenic desi...,0.225000,0.551923,649,84
2,504719,B00000IQCC,A2EGK0YRDF4ZZB,11,11,1018310400,Moving story of Jesus's message in a modern re...,J&eacute;sus de Montr&eacute;al was a stunni...,NaN,moving story jesuss message modern reincarnati...,0.104454,0.523987,4189,547
3,1622425,B00B5UBDA0,A21I62TCDL4754,0,0,1372118400,Paul Anka has aged well and performs well,I was looking for Blu Ray concerts for my new ...,NaN,paul anka aged well performs well looking blu ...,0.468182,0.602273,133,23
4,482286,6305892946,A1IQ9E6I3PIUFF,1,2,969321600,Great end to the series.Or is it???????????,Despite what everyone else says this movie is ...,NaN,great end seriesor despite everyone else says ...,0.800000,0.750000,105,17


In [ ]:
!pip install textstat
from textstat import flesch_reading_ease, smog_index, gunning_fog

data['flesch_reading_ease'] = data['Text'].apply(flesch_reading_ease)
data['smog_index'] = data['Text'].apply(smog_index)
data['gunning_fog'] = data['Text'].apply(gunning_fog)
data.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Review,polarity,subjectivity,char_count,word_count,flesch_reading_ease,smog_index,gunning_fog
0,1224650,B001OQCV6A,A3HUN3KP383B23,0,1,1395619200,Sherlock Holmes,I bought this thinking it was Sherlock Holmes ...,NaN,sherlock holmes bought thinking sherlock holme...,0.500000,0.800000,176,24,73.00,0.0,12.80
1,1019381,B000I8OFLO,A2VJ80PM1G00QV,2,2,1222300800,"FINE SINGING, DESTRUCTIVE STAGING",1. Scenic design and the Directors use of it p...,NaN,fine singing destructive staging 1 scenic desi...,0.225000,0.551923,649,84,61.87,12.3,12.37
2,504719,B00000IQCC,A2EGK0YRDF4ZZB,11,11,1018310400,Moving story of Jesus's message in a modern re...,J&eacute;sus de Montr&eacute;al was a stunni...,NaN,moving story jesuss message modern reincarnati...,0.104454,0.523987,4189,547,57.91,12.5,12.08
3,1622425,B00B5UBDA0,A21I62TCDL4754,0,0,1372118400,Paul Anka has aged well and performs well,I was looking for Blu Ray concerts for my new ...,NaN,paul anka aged well performs well looking blu ...,0.468182,0.602273,133,23,89.58,0.0,7.49
4,482286,6305892946,A1IQ9E6I3PIUFF,1,2,969321600,Great end to the series.Or is it???????????,Despite what everyone else says this movie is ...,NaN,great end seriesor despite everyone else says ...,0.800000,0.750000,105,17,103.63,6.4,4.00


In [ ]:
from collections import Counter
from nltk import pos_tag, word_tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def pos_counts(text):
    tagged = pos_tag(word_tokenize(text))
    counts = Counter(tag for word, tag in tagged)
    return counts.get('NN', 0), counts.get('VB', 0), counts.get('JJ', 0), counts.get('RB', 0)

data['noun_count'], data['verb_count'], data['adj_count'], data['adv_count'] = zip(*data['Text'].apply(pos_counts))
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Review,...,subjectivity,char_count,word_count,flesch_reading_ease,smog_index,gunning_fog,noun_count,verb_count,adj_count,adv_count
0,1224650,B001OQCV6A,A3HUN3KP383B23,0,1,1395619200,Sherlock Holmes,I bought this thinking it was Sherlock Holmes ...,NaN,sherlock holmes bought thinking sherlock holme...,...,0.800000,176,24,73.00,0.0,12.80,4,0,1,4
1,1019381,B000I8OFLO,A2VJ80PM1G00QV,2,2,1222300800,"FINE SINGING, DESTRUCTIVE STAGING",1. Scenic design and the Directors use of it p...,NaN,fine singing destructive staging 1 scenic desi...,...,0.551923,649,84,61.87,12.3,12.37,32,6,12,6
2,504719,B00000IQCC,A2EGK0YRDF4ZZB,11,11,1018310400,Moving story of Jesus's message in a modern re...,J&eacute;sus de Montr&eacute;al was a stunni...,NaN,moving story jesuss message modern reincarnati...,...,0.523987,4189,547,57.91,12.5,12.08,180,24,84,35
3,1622425,B00B5UBDA0,A21I62TCDL4754,0,0,1372118400,Paul Anka has aged well and performs well,I was looking for Blu Ray concerts for my new ...,NaN,paul anka aged well performs well looking blu ...,...,0.602273,133,23,89.58,0.0,7.49,4,0,4,0
4,482286,6305892946,A1IQ9E6I3PIUFF,1,2,969321600,Great end to the series.Or is it???????????,Despite what everyone else says this movie is ...,NaN,great end seriesor despite everyone else says ...,...,0.750000,105,17,103.63,6.4,4.00,6,0,3,2


In [ ]:
data.to_csv('new_data.csv')

In [ ]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 KB 13.2 MB/s eta 0:00:00


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Instantiate the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to extract sentiment features from a given text
def extract_vader_sentiment(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment['pos'], sentiment['neg'], sentiment['neu'], sentiment['compound']

# Apply the function to the text data
data['vader_pos'], data['vader_neg'], data['vader_neu'], data['vader_compound'] = zip(*data['Text'].apply(extract_vader_sentiment))

In [ ]:
data.to_csv('new_test.csv',index=False)

# one-pass tune

In [ ]:
import copy

# Define the initial hyperparameters
default_params = {
    'num_leaves': 100
}
# Define the parameter search space
param_space = {
    'learning_rate': [75,100,125],
}

# Function to calculate the mean squared error of a model
def get_mse(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred)

# Initialize the best parameters and the best MSE with the default parameters
best_params = copy.deepcopy(default_params)
best_mse = get_mse(lgb.LGBMRegressor(**best_params, random_state=42), X_train_combined, y_train, X_test_combined, y_test)

# One-pass search
for param, values in param_space.items():
    for value in values:
        # Update the parameter
        new_params = copy.deepcopy(best_params)
        new_params[param] = value
c
        # Train the model with the new parameter value and calculate the MSE
        new_mse = get_mse(lgb.LGBMRegressor(**new_params, random_state=42), X_train_combined, y_train, X_test_combined, y_test)

        # If the new MSE is better than the best MSE, update the best parameters and the best MSE
        if new_mse < best_mse:
            best_params[param] = value
            best_mse = new_mse
            print(f"Updated best parameters: {best_params}")
            print(f"Updated best MSE: {best_mse:.2f}")

print(f"Final best parameters: {best_params}")
print(f"Final best MSE: {best_mse:.2f}")

